# Dead Reckoning, by hand
This sheet intends to show the steps to go through to calculate Estimated Altitude (He) and Azimuth (Z) from scratch, without navigation tables.  
You might want to get some fter that ;)  
> **_Note_**: To comply with the logarithm tables, we will use here **_decimal_** logathims.  
> natural ones would do to, but this is done for clarity, so you can compare the numbers when using your tables.


The equations to solve are:  
$$
He = arcsin((sin(L).sin(D)) + (cos(L).cos(D).cos(AHL)))
$$

$$
Z = arctan\left(\frac{sin(AHL)}{(cos(D).tan(D))-(sin(L).cos(AHL))}\right)
$$
The required precision implie the usage of logarithms.

In [144]:
import java.math.*;
import java.text.DecimalFormat;

In [145]:
static String decToSex(double v) {
    String s = "";
    double absVal = Math.abs(v);
    double intValue = Math.floor(absVal);
    double dec = absVal - intValue;
    int i = (int) intValue;
    dec *= 60D;
    DecimalFormat df = new DecimalFormat("00.00");
    return Integer.toString(i) + '\272' + df.format(dec) + "'";
}

We start with the following values:  

Estimated position:  
$L = N 37^o 46'$  
$G = W 122^o 37'$

From the almanacs:  
$D = S 6^o 10.84'$  
$AHG = 135^o 03.5'$  
$AHL = 12^o 26.5'$

First we look for the Estimated Altitude He.
$$
He = arcsin((sin(L).sin(D)) + (cos(L).cos(D).cos(AHL)))
$$
We need to find the logarithms of:  
$ sin(L) = sin(37^o46') $  
$ sin(D) = sin(-6^o 10.84')$ rounded to $sin(-6^o 11')$  
$ cos(L) = cos(37^o 46') $. 
$ cos(D) = cos(-6^o 10.84')$ rounded to $cos(-6^o 11')$  
$ cos(AHL) = cos(12^o 26.5') $ rounded to $cos(12^o 26')$  

In [146]:
double lat = 37.0 + (46.0 / 60.0);
double lng = -(122.0 + (37.0 / 60.0));
double dec = -(6.0 + (11.0 / 60.0));
double ahl = 12.0 + (26.0 / 60.0);

### He, Direct Calculation

In [155]:
double directSinHe = (Math.sin(Math.toRadians(lat)) * Math.sin(Math.toRadians(dec))) + (Math.cos(Math.toRadians(lat)) * Math.cos(Math.toRadians(dec)) * Math.cos(Math.toRadians(ahl)));
double heDirect = Math.asin(directSinHe);
System.out.printf("He: %s, Bam!\n", decToSex(Math.toDegrees(heDirect)));

He: 44º32.92', Bam!


java.io.PrintStream@4b32c208

### He, With Logarithms

Notice that we use the absolute values of the angles (for the logaritms to be happy).

In [148]:
double logSinLat = Math.log10(Math.sin(Math.toRadians(Math.abs(lat))));
double logSinDec = Math.log10(Math.sin(Math.toRadians(Math.abs(dec))));
double logCosLat = Math.log10(Math.cos(Math.toRadians(Math.abs(lat))));
double logCosDec = Math.log10(Math.cos(Math.toRadians(Math.abs(dec))));
double logCosAhl = Math.log10(Math.cos(Math.toRadians(Math.abs(ahl))));

In [149]:
System.out.printf("log(sin(L)) = %f\n", logSinLat);
System.out.printf("log(sin(D)) = %f\n", logSinDec);
System.out.printf("log(cos(L)) = %f\n", logCosLat);
System.out.printf("log(cos(D)) = %f\n", logCosDec);
System.out.printf("log(cos(AHL)) = %f\n", logCosAhl);

log(sin(L)) = -0.212931
log(sin(D)) = -0.967743
log(cos(L)) = -0.102092
log(cos(D)) = -0.002534
log(cos(AHL)) = -0.010307


java.io.PrintStream@4b32c208

In [150]:
double logSinLatSinDec = logSinLat + logSinDec;
System.out.printf("log(sin(L).sin(D)) = (%f) + (%f) = %f\n", logSinLat, logSinDec, logSinLatSinDec);

log(sin(L).sin(D)) = (-0.212931) + (-0.967743) = -1.180675


java.io.PrintStream@4b32c208

In [151]:
double sinLsinD = Math.pow(10, logSinLatSinDec);  // exponential base 10
System.out.printf("sin(L).sin(D) = %f\n", sinLsinD);

sin(L).sin(D) = 0.065967


java.io.PrintStream@4b32c208

In [152]:
double logCosLCosDCosAHL = logCosLat + logCosDec + logCosAhl;
System.out.printf("log(cos(L).cos(D).cos(AHL)) = (%f) + (%f) + (%f) = %f\n", logCosLat, logCosDec, logCosAhl, logCosLCosDCosAHL);


log(cos(L).cos(D).cos(AHL)) = (-0.102092) + (-0.002534) + (-0.010307) = -0.114933


java.io.PrintStream@4b32c208

In [153]:
double cosLcosDcosAHL = Math.pow(10, logCosLCosDCosAHL); // exponential base 10
System.out.printf("cos(L).cos(D).cos(AHL) = %f\n", cosLcosDcosAHL);

cos(L).cos(D).cos(AHL) = 0.767481


java.io.PrintStream@4b32c208

In [154]:
// this sign evaluation may need some love...
int sign = (Math.abs(lat) == lat) ? 1 : -1;
sign *= (Math.abs(dec) == dec) ? 1 : -1;
sign *= (Math.abs(ahl) == ahl) ? 1 : -1;

double theSum = sinLsinD + (sign * cosLcosDcosAHL);  // SIGN !!!
System.out.printf("sin(L).sin(D) + cos(L).cos(D).cos(AHL) = %f\n", theSum);

System.out.printf("He: %s\n", decToSex(Math.toDegrees(Math.asin(theSum))));

sin(L).sin(D) + cos(L).cos(D).cos(AHL) = -0.701514
He: 44º32.92'


java.io.PrintStream@4b32c208

### Z, direct